In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
*
FROM `perceptive-ivy-290216.f1_api.sprint_lap_time`  A
# WHERE A.Year=2024
# AND A.GP="Monaco Grand Prix"
# AND A.DRIVER='HAM'
ORDER BY LapNumber, LapStartTime
"""
track3=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [4]:
track2=track3[(track3["GP"]=='United States Grand Prix')&(track3["Year"]==2024)]
track2.head()
year=track2['Year'].iloc[0]
gp=track2['GP'].iloc[0]

In [5]:
track2['LapTime']= pd.to_timedelta(track2["LapTime"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_27817/4173085553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  track2['LapTime']= pd.to_timedelta(track2["LapTime"])


In [6]:
# track2['LapTime'].dt.total_seconds().min()*1.07
track2=track2[track2['LapNumber']!=1.0]

In [7]:
quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
quicklaps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
454,0 days 00:49:23.092000,VER,1,0 days 00:01:38.688000,2.0,1.0,NaT,NaT,0 days 00:00:26.929000,0 days 00:00:39.274000,0 days 00:00:32.485000,0 days 00:48:11.333000,0 days 00:48:50.607000,0 days 00:49:23.092000,217.0,180.0,201.0,305.0,True,MEDIUM,5.0,False,Red Bull Racing,0 days 00:47:44.404000,2024-10-19 18:05:30.503,1,1.0,False,,False,True,2024,United States Grand Prix
455,0 days 00:49:23.943000,NOR,4,0 days 00:01:38.583000,2.0,1.0,NaT,NaT,0 days 00:00:26.977000,0 days 00:00:38.879000,0 days 00:00:32.727000,0 days 00:48:12.371000,0 days 00:48:51.250000,0 days 00:49:23.977000,213.0,183.0,205.0,323.0,True,MEDIUM,7.0,False,McLaren,0 days 00:47:45.360000,2024-10-19 18:05:31.459,1,2.0,False,,False,True,2024,United States Grand Prix
456,0 days 00:49:24.741000,RUS,63,0 days 00:01:38.452000,2.0,1.0,NaT,NaT,0 days 00:00:26.586000,0 days 00:00:39.069000,0 days 00:00:32.797000,0 days 00:48:12.901000,0 days 00:48:51.970000,0 days 00:49:24.767000,211.0,180.0,204.0,328.0,True,MEDIUM,7.0,False,Mercedes,0 days 00:47:46.289000,2024-10-19 18:05:32.388,1,3.0,False,,False,True,2024,United States Grand Prix
457,0 days 00:49:25.616000,LEC,16,0 days 00:01:38.655000,2.0,1.0,NaT,NaT,0 days 00:00:26.785000,0 days 00:00:38.936000,0 days 00:00:32.934000,0 days 00:48:13.763000,0 days 00:48:52.699000,0 days 00:49:25.633000,220.0,187.0,204.0,322.0,True,MEDIUM,5.0,False,Ferrari,0 days 00:47:46.961000,2024-10-19 18:05:33.060,1,4.0,False,,False,True,2024,United States Grand Prix
458,0 days 00:49:25.885000,SAI,55,0 days 00:01:38.506000,2.0,1.0,NaT,NaT,0 days 00:00:26.851000,0 days 00:00:38.586000,0 days 00:00:33.069000,0 days 00:48:14.237000,0 days 00:48:52.823000,0 days 00:49:25.892000,205.0,189.0,208.0,333.0,True,MEDIUM,5.0,False,Ferrari,0 days 00:47:47.379000,2024-10-19 18:05:33.478,1,5.0,False,,False,True,2024,United States Grand Prix


In [8]:
transformed_laps = quicklaps.copy()
transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

# order the team from the fastest (lowest median lap time) tp slower
team_order = (
    transformed_laps[["Driver", "LapTime (s)"]].groupby("Driver").median()["LapTime (s)"].sort_values().index
)
print(team_order)

Index(['VER', 'SAI', 'LEC', 'NOR', 'HAM', 'RUS', 'MAG', 'HUL', 'PIA', 'PER',
       'TSU', 'ALB', 'COL', 'GAS', 'LAW', 'OCO', 'STR', 'ALO', 'ZHO', 'BOT'],
      dtype='object', name='Driver')


In [9]:
transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('median')
transformed_laps.tail()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP,LapTime (s),median
5485,0 days 01:17:55.749000,LAW,30,0 days 00:01:39.978000,19.0,1.0,NaT,NaT,0 days 00:00:26.643000,0 days 00:00:39.774000,0 days 00:00:33.561000,0 days 01:16:42.416000,0 days 01:17:22.190000,0 days 01:17:55.751000,210.0,183.0,204.0,327.0,False,MEDIUM,22.0,False,RB,0 days 01:16:15.771000,2024-10-19 18:34:01.870,1,16.0,False,,False,True,2024,United States Grand Prix,99.978,99.9960
5486,0 days 01:17:56.305000,ALB,23,0 days 00:01:39.748000,19.0,1.0,NaT,NaT,0 days 00:00:26.267000,0 days 00:00:39.897000,0 days 00:00:33.584000,0 days 01:16:42.834000,0 days 01:17:22.731000,0 days 01:17:56.315000,210.0,181.0,200.0,343.0,False,MEDIUM,19.0,True,Williams,0 days 01:16:16.557000,2024-10-19 18:34:02.656,1,17.0,False,,False,True,2024,United States Grand Prix,99.748,99.8875
5489,0 days 01:17:58.548000,ALO,14,0 days 00:01:39.586000,19.0,1.0,NaT,NaT,0 days 00:00:26.606000,0 days 00:00:39.515000,0 days 00:00:33.465000,0 days 01:16:45.601000,0 days 01:17:25.116000,0 days 01:17:58.581000,224.0,184.0,202.0,317.0,False,MEDIUM,22.0,False,Aston Martin,0 days 01:16:18.962000,2024-10-19 18:34:05.061,1,18.0,False,,False,True,2024,United States Grand Prix,99.586,100.2480
5492,0 days 01:18:06.217000,BOT,77,0 days 00:01:43.136000,19.0,1.0,NaT,NaT,0 days 00:00:27.933000,0 days 00:00:40.693000,0 days 00:00:34.510000,0 days 01:16:51.035000,0 days 01:17:31.728000,0 days 01:18:06.238000,212.0,182.0,159.0,298.0,False,MEDIUM,25.0,False,Kick Sauber,0 days 01:16:23.081000,2024-10-19 18:34:09.180,1,20.0,False,,False,True,2024,United States Grand Prix,103.136,100.4240
5493,0 days 01:18:04.583000,ZHO,24,0 days 00:01:41.068000,19.0,1.0,NaT,NaT,0 days 00:00:27.842000,0 days 00:00:40.017000,0 days 00:00:33.209000,0 days 01:16:51.354000,0 days 01:17:31.371000,0 days 01:18:04.580000,206.0,185.0,203.0,320.0,False,MEDIUM,19.0,True,Kick Sauber,0 days 01:16:23.515000,2024-10-19 18:34:09.614,1,19.0,False,,False,True,2024,United States Grand Prix,101.068,100.3835


In [10]:
fig_pace=px.box(
    transformed_laps.sort_values(by=["median","LapNumber"]),
    x="Driver",
    y="LapTime (s)",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace Plot for the {} {} Sprint Race</b>".format(year,gp),
    # palette=team_palette,
    # whiskerprops=dict(color="white"),
    # boxprops=dict(edgecolor="white"),
    # medianprops=dict(color="grey"),
    # capprops=dict(color="white"),
    height=700, width=1200,

color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LAW": "#6692FF",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)
fig_pace.update_traces(opacity=1)
fig_pace.update_layout(
    title_x=0.5,
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    margin=dict(l=50, r=5, t=35, b=50)
)
fig_pace.show()

In [11]:
fig_pace.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Driver Pace/Driver Pace Plots/{}/{} {}Sprint Race Driver Pace Plot.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [12]:
#Get MINIMUM OF THE MEDIAN LAP TIMES

fastest_lap = transformed_laps["median"].min()
transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
transformed_laps_relative_median=transformed_laps[["Driver","Delta","Delta_Percent"]].drop_duplicates()
transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
transformed_laps_relative_median

,Driver,Delta,Delta_Percent
454,VER,0.0000,100.0%
458,SAI,0.1160,100.118%
457,LEC,0.1385,100.141%
455,NOR,0.1735,100.177%
459,HAM,0.5905,100.601%
456,RUS,0.6810,100.694%
460,MAG,1.0310,101.05%
462,HUL,1.1870,101.209%
467,PIA,1.2135,101.236%
463,PER,1.4930,101.521%


In [13]:
for index, row in transformed_laps_relative_median.iterrows():
  if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
    transformed_laps_relative_median.loc[index, 'Delta']=0.001
transformed_laps_relative_median

,Driver,Delta,Delta_Percent
454,VER,0.0010,100.0%
458,SAI,0.1160,100.118%
457,LEC,0.1385,100.141%
455,NOR,0.1735,100.177%
459,HAM,0.5905,100.601%
456,RUS,0.6810,100.694%
460,MAG,1.0310,101.05%
462,HUL,1.1870,101.209%
467,PIA,1.2135,101.236%
463,PER,1.4930,101.521%


In [15]:
fig_median=px.bar(
    transformed_laps_relative_median,
    x="Driver",
    y="Delta",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace (Median Lap Times) for the {} {} Sprint Race</b>".format(year,gp),
    text="Delta_Percent",
    height=700, 
    width=1200,
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LAW": "#6692FF",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig_median.update_traces(textposition='outside')

fig_median.update_traces(opacity=0.85)

fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

fig_median.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_median.update_xaxes(title_text='')      # Remove axis titles
fig_median.update_yaxes(title_text='')

fig_median.update_xaxes(
        title_standoff = 75
)

fig_median.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
fig_median.show()

In [16]:
fig_median.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Driver Pace/Median Lap Times/{}/{} {} Sprint Race Driver Median Plot.html".format(year,year,gp),full_html=False, include_plotlyjs='cdn')

In [17]:
#Get MINIMUM OF ALL LAP TIMES
transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('min')

fastest_lap_all = transformed_laps["min_all"].min()
transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
transformed_laps_relative_min_all=transformed_laps[["Driver","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
transformed_laps_relative_min_all

,Driver,Delta_Min_All,Delta_Percent_All
454,VER,0.000,100.0%
458,SAI,0.089,100.091%
467,PIA,0.104,100.107%
457,LEC,0.285,100.292%
455,NOR,0.582,100.597%
456,RUS,0.729,100.748%
459,HAM,0.860,100.882%
463,PER,0.989,101.015%
460,MAG,1.139,101.169%
462,HUL,1.283,101.316%


In [18]:
for index, row in transformed_laps_relative_min_all.iterrows():
  if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
    transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
transformed_laps_relative_min_all

,Driver,Delta_Min_All,Delta_Percent_All
454,VER,0.001,100.0%
458,SAI,0.089,100.091%
467,PIA,0.104,100.107%
457,LEC,0.285,100.292%
455,NOR,0.582,100.597%
456,RUS,0.729,100.748%
459,HAM,0.860,100.882%
463,PER,0.989,101.015%
460,MAG,1.139,101.169%
462,HUL,1.283,101.316%


In [19]:
fig_min=px.bar(
    transformed_laps_relative_min_all,
    x="Driver",
    y="Delta_Min_All",
    color='Driver',
    template="xgridoff",
    title="<b>Driver Pace (Fastest Lap Times) for the {} {} Sprint Race</b>".format(year,gp),
    text="Delta_Percent_All",
    height=700, width=1200,
    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LAW": "#6692FF",                 
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig_min.update_traces(textposition='outside')

fig_min.update_traces(opacity=0.85)

fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

fig_min.update_layout(
    xaxis={'side': 'top'},
    yaxis={'side': 'left'}
)

fig_min.update_xaxes(title_text='')      # Remove axis titles
fig_min.update_yaxes(title_text='')

fig_min.update_xaxes(
        title_standoff = 75
)

fig_min.update_layout(
    title_x=0.5,
    margin=dict(l=60, r=5, t=110, b=30),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_min.update_yaxes(ticksuffix = "  ")

fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

fig_min.show()

In [ ]:
fig_min.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Driver Pace/Fastest Lap/{}/{} {} Sprint Race Driver Fastest Lap Plot.html".format(year, year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi Race Plot

In [7]:
#2024
# race_multi=['Austrian Grand Prix','Miami Grand Prix','Chinese Grand Prix']
#2023
# race_multi=['Austrian Grand Prix','Azerbaijan Grand Prix','Belgian Grand Prix','Qatar Grand Prix','São Paulo Grand Prix','United States Grand Prix']
#2022
# race_multi=['Austrian Grand Prix','Emilia Romagna Grand Prix','São Paulo Grand Prix']
#2021
race_multi=['British Grand Prix','Italian Grand Prix','São Paulo Grand Prix']

year_multi=2021
for i in race_multi:
    track2=track3[(track3["GP"]==i)&(track3["Year"]==year_multi)]
    year=track2['Year'].iloc[0]
    gp=track2['GP'].iloc[0]
    track2['LapTime']= pd.to_timedelta(track2["LapTime"])
    track2=track2[track2['LapNumber']!=1.0]
    quicklaps=track2[track2["LapTime"]<track2['LapTime'].min()*1.07]
    transformed_laps = quicklaps.copy()
    transformed_laps.loc[:, "LapTime (s)"] = quicklaps["LapTime"].dt.total_seconds()

    # order the team from the fastest (lowest median lap time) tp slower
    team_order = (
        transformed_laps[["Driver", "LapTime (s)"]].groupby("Driver").median()["LapTime (s)"].sort_values().index
    )
    print(team_order)
    transformed_laps['median'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('median')
    transformed_laps.tail()
    fig_pace=px.box(
        transformed_laps.sort_values(by=["median","LapNumber"]),
        x="Driver",
        y="LapTime (s)",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace Plot for the {} {} Sprint Race</b>".format(year_multi,i),
        height=700, width=1200,

    color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            }
    )
    fig_pace.update_traces(opacity=1)
    fig_pace.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow",
        margin=dict(l=50, r=5, t=35, b=50)
    )
    fig_pace.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Driver Pace/Driver Pace Plots/{}/{} {} Sprint Race Driver Pace Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')
    #Get MINIMUM OF THE MEDIAN LAP TIMES

    fastest_lap = transformed_laps["median"].min()
    transformed_laps['Delta']= transformed_laps["median"] - (fastest_lap)
    transformed_laps['Delta_Percent']= (transformed_laps["median"]/(fastest_lap)-1)*100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+100
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].round(3)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent'].astype(str)
    transformed_laps['Delta_Percent']=transformed_laps['Delta_Percent']+"%"
    transformed_laps_relative_median=transformed_laps[["Driver","Delta","Delta_Percent"]].drop_duplicates()
    transformed_laps_relative_median=transformed_laps_relative_median.sort_values("Delta")
    transformed_laps_relative_median
    for index, row in transformed_laps_relative_median.iterrows():
        if transformed_laps_relative_median.loc[index, 'Delta']==0.0:
         transformed_laps_relative_median.loc[index, 'Delta']=0.001
    transformed_laps_relative_median
    fig_median=px.bar(
        transformed_laps_relative_median,
        x="Driver",
        y="Delta",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace (Median Lap Times) for the {} {} Sprint Race</b>".format(year_multi,i),
        text="Delta_Percent",
        height=700, 
        width=1200,
        color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            }
    )

    fig_median.update_traces(textposition='outside')

    fig_median.update_traces(opacity=0.85)

    fig_median.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_median.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_median.update_yaxes(range=[transformed_laps_relative_median['Delta'].max()+0.6, 0])

    fig_median.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_median.update_xaxes(title_text='')      # Remove axis titles
    fig_median.update_yaxes(title_text='')

    fig_median.update_xaxes(
            title_standoff = 75
    )

    fig_median.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_median.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)

    fig_median.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Driver Pace/Median Lap Times/{}/{} {} Sprint Race Driver Median Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

    transformed_laps['min_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform('min')

    fastest_lap_all = transformed_laps["min_all"].min()
    transformed_laps['Delta_Min_All']= transformed_laps["min_all"] - (fastest_lap_all)
    transformed_laps['Delta_Percent_All']= (transformed_laps["min_all"]/(fastest_lap_all)-1)*100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+100
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].round(3)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All'].astype(str)
    transformed_laps['Delta_Percent_All']=transformed_laps['Delta_Percent_All']+"%"
    transformed_laps_relative_min_all=transformed_laps[["Driver","Delta_Min_All","Delta_Percent_All"]].drop_duplicates()
    transformed_laps_relative_min_all=transformed_laps_relative_min_all.sort_values("Delta_Min_All")
    transformed_laps_relative_min_all
    for index, row in transformed_laps_relative_min_all.iterrows():
        if transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']==0.0:
         transformed_laps_relative_min_all.loc[index, 'Delta_Min_All']=0.001
    transformed_laps_relative_min_all
    fig_min=px.bar(
        transformed_laps_relative_min_all,
        x="Driver",
        y="Delta_Min_All",
        color='Driver',
        template="xgridoff",
        title="<b>Driver Pace (Fastest Lap Times) for the {} {} Sprint Race</b>".format(year_multi,i),
        text="Delta_Percent_All",
        height=700, width=1200,
        color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "COL": "#37BEDD",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
            }
    )

    fig_min.update_traces(textposition='outside')

    fig_min.update_traces(opacity=0.85)

    fig_min.update_traces( marker_line_color='white',marker_line_width=1.5)

    fig_min.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

    fig_min.update_yaxes(range=[transformed_laps_relative_min_all['Delta_Min_All'].max()+0.5, 0])

    fig_min.update_layout(
        xaxis={'side': 'top'},
        yaxis={'side': 'left'}
    )

    fig_min.update_xaxes(title_text='')      # Remove axis titles
    fig_min.update_yaxes(title_text='')

    fig_min.update_xaxes(
            title_standoff = 75
    )

    fig_min.update_layout(
        title_x=0.5,
        margin=dict(l=60, r=5, t=110, b=30),
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=12,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_min.update_yaxes(ticksuffix = "  ")

    fig_min.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
    fig_min.write_html("/Users/rdesh723/statpulse-html/plots/Sprint Race Driver Pace/Fastest Lap/{}/{} {} Sprint Race Driver Fastest Lap Plot.html".format(year_multi,year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_11976/833092992.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_11976/833092992.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['VER', 'HAM', 'BOT', 'LEC', 'NOR', 'RIC', 'SAI', 'PER', 'ALO', 'VET',
       'TSU', 'GAS', 'OCO', 'RUS', 'RAI', 'STR', 'LAT', 'GIO', 'MSC', 'MAZ'],
      dtype='object', name='Driver')
Index(['BOT', 'VER', 'RIC', 'NOR', 'HAM', 'LEC', 'SAI', 'PER', 'GIO', 'STR',
       'VET', 'ALO', 'OCO', 'RUS', 'LAT', 'TSU', 'MAZ', 'KUB', 'MSC'],
      dtype='object', name='Driver')
Index(['VER', 'BOT', 'HAM', 'SAI', 'PER', 'NOR', 'LEC', 'VET', 'OCO', 'ALO',
       'GAS', 'RIC', 'GIO', 'TSU', 'STR', 'RAI', 'LAT', 'RUS', 'MSC', 'MAZ'],
      dtype='object', name='Driver')


/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_11976/833092992.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# #Get AVERAGE OF ALL LAP TIMES
# transformed_laps['avg_all'] = transformed_laps['LapTime (s)'].groupby(transformed_laps['Driver']).transform(np.mean)

# fastest_lap_avg = transformed_laps["avg_all"].min()
# transformed_laps['Delta_Avg_All']= transformed_laps["avg_all"] - (fastest_lap_avg)
# transformed_laps['Delta_Percent_Avg']= (transformed_laps["avg_all"]/(fastest_lap_avg)-1)*100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+100
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].round(3)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg'].astype(str)
# transformed_laps['Delta_Percent_Avg']=transformed_laps['Delta_Percent_Avg']+"%"
# transformed_laps_relative_avg_all=transformed_laps[["Driver","Delta_Avg_All","Delta_Percent_Avg"]].drop_duplicates()
# transformed_laps_relative_avg_all=transformed_laps_relative_avg_all.sort_values("Delta_Avg_All")
# transformed_laps_relative_avg_all

In [201]:
# for index, row in transformed_laps_relative_avg_all.iterrows():
#   if transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']==0.0:
#     transformed_laps_relative_avg_all.loc[index, 'Delta_Avg_All']=0.001
# transformed_laps_relative_avg_all

In [202]:
# fig=px.bar(
#     transformed_laps_relative_avg_all,
#     x="Driver",
#     y="Delta_Avg_All",
#     color='Driver',
#     template="xgridoff",
#     title="<b>Driver Pace (Average Lap Times) for the {} {}</b>".format(year,gp),
#     text="Delta_Percent_Avg",
#     height=700, width=1200,
#     color_discrete_map={
#                  "ALB": "#64C4FF",
#                  "ALO": "#229971",
#                  "BOT": "#52e252",
#                  "GAS": "#0093cc",
#                  "HAM": "#27F4D2",
#                  "HUL": "#B6BABD",
#                  "LEC": "#E80020",
#                  "MAG": "#B6BABD",
#                  "NOR": "#FF8000",
#                  "OCO": "#0093cc",
#                  "PER": "#3671C6",
#                  "PIA": "#FF8000",
#                  "RIC": "#6692FF",
#                  "RUS": "#27F4D2",
#                  "SAI": "#E80020",
#                  "SAR": "#37BEDD",
#                  "STR": "#229971",
#                  "TSU": "#6692FF",
#                  "VER": "#3671C6",
#                  "ZHO": "#52e252"
#           }
# )

# fig.update_traces(textposition='outside')

# fig.update_traces(opacity=0.85)

# fig.update_traces( marker_line_color='white',marker_line_width=1.5)

# fig.update_yaxes(tickangle = -90,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_xaxes(tickangle = -0,tickprefix="<b>",ticksuffix ="</b><br>")

# fig.update_yaxes(range=[3.5, 0])

# fig.update_layout(
#     xaxis={'side': 'top'},
#     yaxis={'side': 'left'}
# )

# fig.update_xaxes(title_text='')      # Remove axis titles
# fig.update_yaxes(title_text='')

# fig.update_xaxes(
#         title_standoff = 75
# )

# fig.update_layout(
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     title_x=0.5,
#     margin=dict(l=60, r=5, t=110, b=30),
#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=12,
#         color="Black"
#     ),
#     title_font_family="PT Sans Narrow"
# )
# fig.update_traces( marker_line_color='Black',marker_line_width=0.1, opacity=0.99)
# fig.show()

In [166]:
# fig.write_html("{} {} Driver Avg Lap Plot.html".format(year,gp))